## 1. Import Libraries:

In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

## 2. Load Datasets:

In [72]:
df_train = pd.read_csv('../C_Datasets/titanic_train_original.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [73]:
df_test = pd.read_csv('../C_Datasets/titanic_test_original.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## 2. Create Custom Functions:

In [67]:
def dropper(X):
    return X.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

def imputer(X):
    copy_X = X.copy()
    imp = SimpleImputer(strategy='mean')
    copy_X[['Age', 'Fare']] = imp.fit_transform(copy_X[['Age', 'Fare']])
    return copy_X

def dummies(X):
    return pd.get_dummies(data=X, columns=['Embarked'], drop_first=True)

def le(X):
    copy_X = X.copy()
    le = LabelEncoder()
    copy_X['Sex'] = le.fit_transform(copy_X['Sex'])
    return copy_X

def scaler(X):
    copy_X = X.copy()
    scaler = StandardScaler()
    copy_X[['Age', 'Fare']] = scaler.fit_transform(copy_X[['Age', 'Fare']])
    return copy_X

## 3. Wrap the Functions Using ```'FunctionTransformer'```:

In [68]:
Dropper = FunctionTransformer(dropper)
Imputer = FunctionTransformer(imputer)
GetDummies = FunctionTransformer(dummies)
LE = FunctionTransformer(le)
Scaler = FunctionTransformer(scaler)

## 4. Create a Pipeline for Preprocessin:

In [69]:
pipe = Pipeline([
    ('dropper', Dropper),  # Step 1: Apply log transformation
    ('imputer', Imputer),  # Step 2: Standardize the data
    ('dummies', GetDummies),  # Step 3: Apply SVM
    ('LE', LE),
    ('Scaler', Scaler)
])

## 5. Fit the Pipeline for Train Set:

In [74]:
df_train = pipe.fit_transform(df_train)

In [75]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,1,-0.592481,1,0,-0.502445,0,1
1,1,1,0,0.638789,1,0,0.786845,0,0
2,1,3,0,-0.284663,0,0,-0.488854,0,1
3,1,1,0,0.407926,1,0,0.420730,0,1
4,0,3,1,0.407926,0,0,-0.486337,0,1


## 6. Fit the Pipeline for Test Set:

In [76]:
df_test = pipe.fit_transform(df_test)

In [77]:
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,1,0.334993,0,0,-0.498407,1,0
1,3,0,1.325530,1,0,-0.513274,0,1
2,2,1,2.514175,0,0,-0.465088,1,0
3,3,1,-0.259330,0,0,-0.483466,0,1
4,3,0,-0.655545,1,1,-0.418471,0,1
